In [ ]:
import torchcodec
print(torchcodec.__version__)

0.9.1


In [2]:
from datasets import Audio
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", split="validation")

ds = ds.cast_column("audio", Audio(sampling_rate=16000))
sample = ds[0]["audio"]["array"]

print(sample.shape)


/home/bishwa/Unversity/Audio_LLM_Memory/audio_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 73/73 [00:00<00:00, 1743.44 examples/s]

(93680,)


In [8]:
ds[0]['audio']['array']

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], shape=(93680,), dtype=float32)

In [3]:
sample

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], shape=(93680,), dtype=float32)

In [2]:
import torch
from torch import nn
from fla.layers.mom import MomAttention as Mom

In [3]:
batch_size = 2
seq_len    = 64
d_model    = 256      # hidden dimension
n_mem      = 8        # ← number of memory slots (most important MoM param)
dropout    = 0.1

In [12]:
mom_layer = Mom(
    hidden_size=d_model          
)

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
mom_layer = mom_layer.to(device=device)

In [19]:
x = torch.randn(batch_size, seq_len, d_model, device=device)

In [20]:
x.shape

torch.Size([2, 64, 256])

In [21]:
mom_layer(x)

AttributeError: module 'torch.cpu' has no attribute 'device'

In [ ]:
         # the main layer

# ─── Hyperparameters ───────────────────────────────────────


device = "cuda" if torch.cuda.is_available() else "cpu"

# ─── Create MoM layer ──────────────────────────────────────
# You can plug almost any linear mixing function; here we use simple linear attention style
mom_layer = Mom(
    d_model=d_model,
    n_mem=n_mem,               # how many parallel memory states
    dropout=dropout,
    # You can also choose different mixing_fn e.g. 'linear', 'delta_rule', etc.
    # mixing_fn='linear',      # default in many cases
    # bias=True,
    # norm_type='layernorm',
).to(device)

# Optional: wrap in a tiny model (MoM is usually used like an RNN / linear attn block)
class TinyMoMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mom = mom_layer
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, d_model)   # or vocab size in real LM

    def forward(self, x):
        # x : (batch, seq, dim)
        x = self.mom(x)          # ← core call
        x = self.norm(x)
        return self.head(x)

model = TinyMoMModel().to(device)

# ─── Dummy data ─────────────────────────────────────────────
# Random float sequences (in real life → token embeddings)
x = torch.randn(batch_size, seq_len, d_model, device=device)

# Optional: attention mask / key padding mask (usually causal for autoregressive)
# For simplicity we use full attention here → set causal=True if needed in real usage
mask = None   # or torch.ones(batch_size, seq_len, device=device) for full mask

# ─── Forward pass ───────────────────────────────────────────
out = model(x)                      # shape: (batch, seq, d_model)

print("Input shape :", x.shape)
print("Output shape:", out.shape)   # should match input shape
print(f"Memory usage roughly scales as O(batch × seq × d_model) — not quadratic!")

# Quick gradient check
out.sum().backward()
print("Grad check ok ✓")